In [20]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from coco_data_loader import get_validation_loader 
from coco_build_vocab import Vocabulary
from coco_model import EncoderCNN, DecoderRNNWithAttention
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

def main(args):
    # Image preprocessing
    transform = transforms.Compose([
        transforms.Resize((args.image_size, args.image_size)),
        transforms.ToTensor(), 
     #   transforms.Normalize((0.485, 0.456, 0.406), 
      #                       (0.229, 0.224, 0.225))
    ])
    
    # Load vocabulary wrapper
    with open(args.vocab_path, 'rb') as f:
        vocab = pickle.load(f)
        
    # Build data loader
    data_loader = get_validation_loader(args.image_dir, args.caption_path, vocab, 
                             transform, args.batch_size,
                             num_workers=args.num_workers)
    
    # Build models
    encoder = EncoderCNN(args.encoded_image_size).eval()  # eval mode (batchnorm uses moving mean/variance)
    decoder = DecoderRNNWithAttention(args.embed_size, args.attention_size, args.hidden_size, len(vocab)).eval()
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    # Load the trained model parameters
    encoder.load_state_dict(torch.load(args.encoder_path))
    decoder.load_state_dict(torch.load(args.decoder_path))
    
    ground_truth = []
    predicted = []
    for i, (images, captions) in enumerate(data_loader):
        # Set mini-batch dataset
        images = images.to(device)
        features = encoder(images)
        sampled_seq = decoder.sample_beam_search(features, vocab, device)
        ground_truth.append(captions[0])
        predicted.append(sampled_seq[0])
        if i > 10: break
            
    print(corpus_bleu(ground_truth, predicted, weights=(1, 0, 0, 0)))
    print(corpus_bleu(ground_truth, predicted, weights=(0.5, 0.5, 0, 0)))
    print(corpus_bleu(ground_truth, predicted, weights=(1.0/3.0, 1.0/3.0, 1.0/3.0, 0)))
    print(corpus_bleu(ground_truth, predicted))

if __name__ == '__main__':
    import sys
    sys.argv = ['foo']
    parser = argparse.ArgumentParser()
    parser.add_argument('--encoder_path', type=str, default='./coco_models/encoder-9-2232.ckpt', help='path for trained encoder')
    parser.add_argument('--decoder_path', type=str, default='./coco_models/decoder-9-2232.ckpt', help='path for trained decoder')
    parser.add_argument('--vocab_path', type=str, default='./data/coco_vocab.pkl', help='path for vocabulary wrapper')
    parser.add_argument('--image_dir', type=str, default='../mscoco/resized_train2014', help='directory for resized images')
    parser.add_argument('--caption_path', type=str, default='../mscoco/annotations/captions_train2014.json', help='path for train annotation json file')
    parser.add_argument('--image_size', type=int , default=224, help='input image size')

    # Model parameters (should be same as paramters in train.py)
    parser.add_argument('--embed_size', type=int , default=256, help='dimension of word embedding vectors')
    parser.add_argument('--encoded_image_size', type=int , default=14, help='dimension of encoded image')
    parser.add_argument('--attention_size', type=int , default=384, help='dimension of attention layers')
    parser.add_argument('--hidden_size', type=int , default=384, help='dimension of lstm hidden states')

    parser.add_argument('--batch_size', type=int, default=1)
    parser.add_argument('--num_workers', type=int, default=4)

    args = parser.parse_args()
    print(args)
    main(args)


cuda:1
Namespace(attention_size=384, batch_size=1, caption_path='../mscoco/annotations/captions_train2014.json', decoder_path='./coco_models/decoder-9-2232.ckpt', embed_size=256, encoded_image_size=14, encoder_path='./coco_models/encoder-9-2232.ckpt', hidden_size=384, image_dir='../mscoco/resized_train2014', image_size=224, num_workers=4, vocab_path='./data/coco_vocab.pkl')
loading annotations into memory...
Done (t=0.79s)
creating index...
index created!
0.6111111111111112
0.4194352464039306
0.24474633652528716
0.16140750043829394
